In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [ ]:
ukb37332 = pd.read_sas('SAS\\UKBIOBANK\\ukb37332.sas7bdat', format = 'sas7bdat')
ukb37332

In [ ]:
ukb45783_data = []
ukb45783 = pd.read_sas('SAS\\UKBIOBANK\\ukb45783.sas7bdat', format = 'sas7bdat', chunksize=100000, iterator=True)

for chunk in ukb45783:
    ukb45783_data.append(chunk)
asm_data_df=pd.concat(ukb45783_data)

In [ ]:
ukb46987 = pd.read_sas('SAS\\UKBIOBANK\\ukb46987.sas7bdat', format = 'sas7bdat')
ukb46987

#### ukb46987_new에서 ckd 진단자, ckd 진단일 열 생성

In [3]:
ukb46987 = pd.read_sas('SAS\\ukb46987_new.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
ukb46987

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = ukb46987.copy()

In [ ]:
df2 = df.iloc[:, :224]
df2

In [ ]:
df4 = df.iloc[:, 224:]
df4

In [ ]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# n18 = 0
# for row in range(len(df2)): 
#     for col in colnames: 
#         print(df2[col][row])
#         if df2[col][row] is not np.nan: 
#             disease = df2[col][row].decode('euc-kr')
#             if disease.startswith('N18') == True: 
#                 n18 += 1
#                 break

In [ ]:
df2['N180'] = 0
df2['N181'] = 0
df2['N182'] = 0
df2['N183'] = 0
df2['N184'] = 0
df2['N185'] = 0
df2['N186'] = 0
df2['N187'] = 0
df2['N188'] = 0
df2['N189'] = 0

In [ ]:
df2

In [ ]:
colnames = df2.columns.tolist()
colnames.remove('eid')

for col in colnames: 
    df2.loc[df2[col] == 'N180', 'N180'] = 1  # 전체 222개 칼럼 순회하면서 N180 있으면 1 넣어주기
    df2.loc[df2[col] == 'N180', 'N180_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

    #df2["N180"] = df2.apply(lambda x: 1 if x[col] == 'N180' else 0, axis=0)

In [ ]:
df2['N180'].value_counts() 

In [ ]:
# 인덱스랑 진단여부 갯수 똑같은 거 확인

df2['N180_index'].isna().sum()

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N181', 'N181'] = 1  # 전체 222개 칼럼 순회하면서 N181 있으면 1 넣어주기
    df2.loc[df2[col] == 'N181', 'N181_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N182', 'N182'] = 1  # 전체 222개 칼럼 순회하면서 N182 있으면 1 넣어주기
    df2.loc[df2[col] == 'N182', 'N182_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N183', 'N183'] = 1  # 전체 222개 칼럼 순회하면서 N183 있으면 1 넣어주기
    df2.loc[df2[col] == 'N183', 'N183_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N184', 'N184'] = 1  # 전체 222개 칼럼 순회하면서 N184 있으면 1 넣어주기
    df2.loc[df2[col] == 'N184', 'N184_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N185', 'N185'] = 1  # 전체 222개 칼럼 순회하면서 N185 있으면 1 넣어주기
    df2.loc[df2[col] == 'N185', 'N185_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N186', 'N186'] = 1  # 전체 222개 칼럼 순회하면서 N186 있으면 1 넣어주기
    df2.loc[df2[col] == 'N186', 'N186_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N187', 'N187'] = 1  # 전체 222개 칼럼 순회하면서 N187 있으면 1 넣어주기
    df2.loc[df2[col] == 'N187', 'N187_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N188', 'N188'] = 1  # 전체 222개 칼럼 순회하면서 N188 있으면 1 넣어주기
    df2.loc[df2[col] == 'N188', 'N188_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N189', 'N189'] = 1  # 전체 222개 칼럼 순회하면서 N189 있으면 1 넣어주기
    df2.loc[df2[col] == 'N189', 'N189_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# for col in colnames: 
#     print(col, ' ', df2.loc[df2[col] == 'N181', col].count())

In [ ]:
# for col in colnames: 
#     df["N180"] = df.apply(lambda x: 1 if x[col] == 'N180' else 0, axis = 1)
#     df["N181"] = df.apply(lambda x: 1 if x[col] == 'N181' else 0, axis = 1)
#     df["N182"] = df.apply(lambda x: 1 if x[col] == 'N182' else 0, axis = 1)
#     df["N183"] = df.apply(lambda x: 1 if x[col] == 'N183' else 0, axis = 1)
#     df["N184"] = df.apply(lambda x: 1 if x[col] == 'N184' else 0, axis = 1)
#     df["N185"] = df.apply(lambda x: 1 if x[col] == 'N185' else 0, axis = 1)
#     df["N186"] = df.apply(lambda x: 1 if x[col] == 'N186' else 0, axis = 1)
#     df["N187"] = df.apply(lambda x: 1 if x[col] == 'N187' else 0, axis = 1)
#     df["N188"] = df.apply(lambda x: 1 if x[col] == 'N188' else 0, axis = 1)
#     df["N189"] = df.apply(lambda x: 1 if x[col] == 'N189' else 0, axis = 1)

In [ ]:
df2["N180"].value_counts()

In [ ]:
df2['N180_index'].isna().sum()

In [ ]:
df2["N181"].value_counts()

In [ ]:
df2['N181_index'].isna().sum()

In [ ]:
df2["N182"].value_counts()

In [ ]:
df2['N182_index'].isna().sum()

In [ ]:
df2["N183"].value_counts()

In [ ]:
df2['N183_index'].isna().sum()

In [ ]:
df2["N184"].value_counts()

In [ ]:
df2['N184_index'].isna().sum()

In [ ]:
df2["N185"].value_counts()

In [ ]:
df2['N185_index'].isna().sum()

In [ ]:
df2["N186"].value_counts()

In [ ]:
df2['N186_index'].isna().sum()

In [ ]:
df2["N187"].value_counts()

In [ ]:
df2['N187_index'].isna().sum()

In [ ]:
df2["N188"].value_counts()

In [ ]:
df2['N188_index'].isna().sum()

In [ ]:
df2["N189"].value_counts()

In [ ]:
df2['N189_index'].isna().sum()

In [ ]:
df_final = df2[['eid', 'N180', 'N181', 'N182', 'N183', 'N184', 'N185', 'N186', 'N187', 'N188', 'N189', 
                'N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']]
df_final

In [ ]:
df4

In [ ]:
df5 = pd.concat([df_final, df4], axis = 1)
df5

In [ ]:
dftmp = df5[['N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']].copy()

In [ ]:
dftmp['N180_index'].unique()

In [ ]:
#### TEST

coltest = ['idx1', 'idx2', 'idx3', '41280-1.0', '41280-5.0', '41280-9.0']
value = [[1, np.nan, np.nan, '2010-01-01', np.nan, np.nan], [np.nan, 5, np.nan, np.nan, '2010-03-03', np.nan], [np.nan, np.nan, 9, np.nan, np.nan, '2021-01-01'], [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]]
testdf = pd.DataFrame(columns = coltest, data=value)

# vltest = testdf[col].sum(axis = 1).tolist()
# vltestdf = pd.DataFrame(vltest)
# vltestdf[0].unique()

testdf['N18_idx'] = testdf.stack().groupby(level=0).first().reindex(testdf.index)   # 각 행에서 Nan 값이 아닌 유일한 값 가져오기
testdf
# testdf['N18_Date'] = list(map(lambda y: np.nan if y is np.nan else testdf['41280-' + str(y)].values[0], testdf['N18_idx']))

In [ ]:
testdf['eid'] = ['A', 'B', 'C', 'D']
valuelist = testdf['N18_idx'].tolist()

testdf['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else testdf.loc[testdf['eid'] == x, '41280-' + str(y)].values[0], testdf['eid'], valuelist))

In [ ]:
testdf

In [ ]:
########## test end

In [ ]:
column_list = ['N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']

dftmp['N18_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

In [ ]:
dftmp['N18_idx'].unique()

In [ ]:
dftmp

In [ ]:
df5 = pd.concat([df5, dftmp['N18_idx']], axis = 1)
df5

In [ ]:
c = df5.columns.tolist()
for col in c: 
    print(col)

In [ ]:
# 열별로 진단받은 날짜 추가하기

valuelist = df5['N18_idx'].tolist()
df5['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else df5.loc[df5['eid'] == x, '41280-' + str(y)].values[0], df5['eid'], valuelist))

In [ ]:
len(df5['N18_Date'].unique())

In [ ]:
df5

In [ ]:
df6 = df5[['eid', 
'N180', 'N181', 'N182', 'N183', 'N184', 'N185', 'N186', 'N187', 'N188', 'N189', 
'N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index',
'N18_idx', 'N18_Date']]

In [ ]:
df6

In [ ]:
df6.to_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.06 ~

In [ ]:
df = pd.read_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr')
df

In [ ]:
def conditions(df):
    if (df['N180'] == 1) or (df['N181'] == 1) or (df['N182'] == 1) or (df['N183'] == 1) or (df['N184'] == 1) or (df['N185'] == 1) or (df['N186'] == 1) or (df['N187'] == 1) or (df['N188'] == 1) or (df['N189'] == 1):
        return 1
    else:
        return 0

In [ ]:
df['N18'] = df.apply(conditions, axis=1)

In [ ]:
df['N18'].value_counts()

In [ ]:
df['N18_Date'].fillna(0, inplace = True)

In [ ]:
df.loc[df['N18_Date'] == 0, 'N18_Date'].count()

In [ ]:
df

In [ ]:
df_final = df[['eid', 'N18_Date', 'N18']]
df_final

In [ ]:
df_final.to_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr', index = False)

#### HTN drug preprocessing

In [ ]:
ukb45783_drug = pd.read_sas('SAS\\ukb45783_drug.sas7bdat', format = 'sas7bdat')
ukb45783_drug

In [ ]:
df = ukb45783_drug.copy()

In [ ]:
HTN_drugcode_preprocessed_FINAL = pd.read_csv('data\\HTN_drugcode_preprocessed_FINAL.csv', encoding = 'euc-kr')
HTN_drugcode_preprocessed_FINAL

In [ ]:
htn_drugcode = HTN_drugcode_preprocessed_FINAL['UK Biobank code'].dropna()
htn_drugcode = htn_drugcode.tolist()
htn_drugcode

In [ ]:
colnames = df.columns.tolist()
colnames.remove('eid')

In [ ]:
for col in colnames: 
    for val in htn_drugcode: 
        df.loc[df[col] == val, str(val)] = 1  # 전체 칼럼 순회하면서 drug code 있으면 칼럼 만들어서 1 넣어주기

In [ ]:
df

In [ ]:
# 엑셀로 정리해둔 명수랑 얼추 맞음

for val in htn_drugcode: 
    print(df[str(val)].value_counts())

In [ ]:
htn_drugcode_str = list(map(str, htn_drugcode))
htn_drugcode_str.insert(0, 'eid')
htn_drugcode_str

In [ ]:
df_final = df[htn_drugcode_str].copy()
df_final

In [ ]:
df_final.fillna(0, inplace = True)
df_final

In [ ]:
#### test

d = {'A':[1, 0, 0, 0, 0], 
          'B':[0, 1, 0, 0, 1], 
          'C':[1, 1, 1, 0, 0], 
          'D':[0, 0, 0, 0, 1], 
          'E':[0, 0, 0, 0, 0], 
          'col':[1, 1, 1, 0, 1]}
testdf = pd.DataFrame(data = d, index = ['prsn1', 'prsn2', 'prsn3', 'prsn4', 'prsn5'])
testdf

In [ ]:
def create_htnmed(df):
    count = 0
    col = pd.DataFrame(df).columns.tolist()
    print(col)

    for c in col: 
        if df[str(c)] == 1: 
            count += 1
    if count > 0: 
        return 1
    else: 
        return 0

In [ ]:
for val in htn_drugcode: 
    df_final.loc[df_final[str(val)] == 1, 'HTN_med'] = 1

In [ ]:
df_final['HTN_med'].fillna(0, inplace = True)

In [ ]:
df_final['HTN_med'].value_counts()

In [ ]:
df_final

In [ ]:
df_final.to_csv('data\\ukb45783_drugpreprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.08 ~     
#### Last follow up date 생성

In [2]:
# 답 없음 eid는 num으로, date는 전부 character로 만들어서 sas로 가져가자

date_merged = pd.read_sas('SAS\\ukb_date_merged.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
date_merged

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,...,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,last_followup_date
0,NaN,NaN,2014-11-08,NaN,2014-11-08,NaN,2014-11-08,NaN,2016-01-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,2016-10-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2014-12-01,NaN,2014-12-01,NaN,2014-12-01,NaN,2017-01-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2015-07-12,NaN,2015-07-12,NaN,2015-07-12,NaN,2017-03-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2015-01-31,NaN,2015-01-31,NaN,2015-01-31,NaN,2017-08-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,2014-11-29,NaN,2014-11-29,NaN,2014-11-29,NaN,2016-05-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,2015-02-07,NaN,2015-02-07,NaN,2015-02-07,NaN,2017-04-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,2015-02-21,NaN,2015-02-21,NaN,2015-02-21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,2015-09-17,NaN,2015-09-17,NaN,2015-09-17,NaN,2016-12-20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# null을 다 1900-01-01로 채우고, 날짜 -> 문자로 바꿔서 sas에서 가져온 데이터셋 

date_changed = pd.read_sas('SAS\\UKB_DATE_MERGED_DATETIME2.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
date_changed

,eid,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1000015.0,1900-01-01,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2016-01-10,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
1,1000027.0,1900-01-01,1900-01-01,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
2,1000039.0,1900-01-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2017-01-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,1000040.0,1900-01-01,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2017-03-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,1000053.0,1900-01-01,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2017-08-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,6025327.0,1900-01-01,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2016-05-10,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502471,6025339.0,1900-01-01,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2017-04-06,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502472,6025340.0,1900-01-01,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502473,6025353.0,1900-01-01,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2016-12-20,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01


In [4]:
df = date_changed.copy()

In [5]:
id_tmp = df[['eid']]

In [6]:
id_tmp

,eid
0,1000015.0
1,1000027.0
2,1000039.0
3,1000040.0
4,1000053.0
...,...
502470,6025327.0
502471,6025339.0
502472,6025340.0
502473,6025353.0


In [7]:
df.drop(['eid'], axis = 1, inplace = True)
df

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1900-01-01,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2016-01-10,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
1,1900-01-01,1900-01-01,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,2016-10-20,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
2,1900-01-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2017-01-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,1900-01-01,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2017-03-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,1900-01-01,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2017-08-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,1900-01-01,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2016-05-10,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502471,1900-01-01,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2017-04-06,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502472,1900-01-01,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,1900-01-01,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502473,1900-01-01,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2016-12-20,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01


In [ ]:
df['30502-0.0'][0]

In [34]:
# Nan이 있으면 max value가 다 nan으로 잡힌다

df.fillna('0', inplace = True)

In [51]:
# datetime 형식으로 전부 바꾸기

df.iloc[:, 0:493] = df.iloc[:, 0:493].apply(pd.to_datetime, errors='coerce')

In [ ]:
# df.replace(0, '0', inplace = True)

In [52]:
df

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,30611-0.0,30611-1.0,30621-0.0,30621-1.0,30631-0.0,30631-1.0,30651-0.0,30651-1.0,30661-0.0,30661-1.0,30671-0.0,30671-1.0,30681-0.0,30681-1.0,30691-0.0,30691-1.0,30701-0.0,30701-1.0,30711-0.0,30711-1.0,30721-0.0,30721-1.0,30731-0.0,30731-1.0,30741-0.0,30741-1.0,30751-0.0,30751-1.0,30761-0.0,30761-1.0,30771-0.0,30771-1.0,30781-0.0,30781-1.0,30791-0.0,30791-1.0,30801-0.0,30801-1.0,30811-0.0,30811-1.0,30821-0.0,30821-1.0,30831-0.0,30831-1.0,30841-0.0,30841-1.0,30851-0.0,30851-1.0,30861-0.0,30861-1.0,30871-0.0,30871-1.0,30881-0.0,30881-1.0,30891-0.0,30891-1.0,53-0.0,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,22700-0.0,22700-0.1,22700-0.2,22700-0.3,22700-0.4,22700-0.5,22700-0.6,22700-0.7,22700-0.8,22700-0.9,22700-0.10,22700-0.11,22700-0.12,22700-0.13,22700-0.14,30532-0.0,30532-1.0,40000-0.0,40000-1.0,40005-0.0,40005-1.0,40005-2.0,40005-3.0,40005-4.0,40005-5.0,40005-6.0,40005-7.0,40005-8.0,40005-9.0,40005-10.0,40005-11.0,40005-12.0,40005-13.0,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0,41262-0.0,41262-0.1,41262-0.2,41262-0.3,41262-0.4,41262-0.5,41262-0.6,41262-0.7,41262-0.8,41262-0.9,41262-0.10,41262-0.11,41262-0.12,41262-0.13,41262-0.14,41262-0.15,41262-0.16,41262-0.17,41262-0.18,41262-0.19,41262-0.20,41262-0.21,41262-0.22,41262-0.23,41262-0.24,41262-0.25,41262-0.26,41262-0.27,41262-0.28,41262-0.29,41262-0.30,41262-0.31,41262-0.32,41262-0.33,41262-0.34,41262-0.35,41262-0.36,41262-0.37,41262-0.38,41262-0.39,41262-0.40,41262-0.41,41262-0.42,41262-0.43,41262-0.44,41262-0.45,41262-0.46,41262-0.47,41262-0.48,41262-0.49,41262-0.50,41262-0.51,41262-0.52,41262-0.53,41262-0.54,41262-0.55,41262-0.56,41262-0.57,41262-0.58,41262-0.59,41262-0.60,41262-0.61,41262-0.62,41262-0.63,41262-0.64,41262-0.65,41262-0.66,41262-0.67,41262-0.68,41262-0.69,41262-0.70,41262-0.71,41262-0.72,41262-0.73,41262-0.74,41263-0.0,41263-0.1,41263-0.2,41263-0.3,41263-0.4,41263-0.5,41263-0.6,41263-0.7,41263-0.8,41263-0.9,41263-0.10,41263-0.11,41263-0.12,41263-0.13,41263-0.14,41263-0.15,41263-0.16,41263-0.17,41263-0.18,41263-0.19,41263-0.20,41263-0.21,41263-0.22,41263-0.23,41263-0.24,41263-0.25,41263-0.26,41263-0.27,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,41280-0.10,41280-0.11,41280-0.12,41280-0.13,41280-0.14,41280-0.15,41280-0.16,41280-0.17,41280-0.18,41280-0.19,41280-0.20,41280-0.21,41280-0.22,41280-0.23,41280-0.24,41280-0.25,41280-0.26,41280-0.27,41280-0.28,41280-0.29,41280-0.30,41280-0.31,41280-0.32,41280-0.33,41280-0.34,41280-0.35,41280-0.36,41280-0.37,41280-0.38,41280-0.39,41280-0.40,41280-0.41,41280-0.42,41280-0.43,41280-0.44,41280-0.45,41280-0.46,41280-0.47,41280-0.48,41280-0.49,41280-0.50,41280-0.51,41280-0.52,41280-0.53,41280-0.54,41280-0.55,41280-0.56,41280-0.57,41280-0.58,41280-0.59,41280-0.60,41280-0.61,41280-0.62,41280-0.63,41280-0.64,41280-0.65,41280-0.66,41280-0.67,41280-0.68,41280-0.69,41280-0.70,41280-0.71,41280-0.72,41280-0.73,41280-0.74,41280-0.75,41280-0.76,41280-0.77,41280-0.78,41280-0.79,41280-0.80,41280-0.81,41280-0.82,41280-0.83,41280-0.84,41280-0.85,41280-0.86,41280-0.87,41280-0.88,41280-0.89,41280-0.90,41280-0.91,41280-0.92,41280-0.93,41280-0.94,41280-0.95,41280-0.96,41280-0.97,41280-0.98,41280-0.99,41280-0.100,41280-0.101,41280-0.102,41280-0.103,41280-0.104,41280-0.105,41280-0.106,41280-0.107,41280-0.108,41280-0.109,41280-0.110,41280-0.111,41280-0.112,41280-0.113,41280-0.114,41280-0.115,41280-0.116,41280-0.117,41280-0.118,41280-0.119,41280-0.120,41280-0.121,41280-0.122,41280-0.123,41280-0.124,41280-0.125,41280-0.126,41280-0.127,41280-0.128,41280-0.129,41280-0.130,41280-0.131,41280-0.132,41280-0.133,41280-0.134,41280-0.135,41280-0.136,41280-0.137,41280-0.138,41280-0.139,41280-0.140,41280-0.141,41280-0.142,41280-0.143,41280-0.144,41280-0.145,41280-0.146,41280-0.147,41280-0.148,41280-0.

In [53]:
df = pd.concat([id_tmp, df], axis = 1)
df

,eid,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,30611-0.0,30611-1.0,30621-0.0,30621-1.0,30631-0.0,30631-1.0,30651-0.0,30651-1.0,30661-0.0,30661-1.0,30671-0.0,30671-1.0,30681-0.0,30681-1.0,30691-0.0,30691-1.0,30701-0.0,30701-1.0,30711-0.0,30711-1.0,30721-0.0,30721-1.0,30731-0.0,30731-1.0,30741-0.0,30741-1.0,30751-0.0,30751-1.0,30761-0.0,30761-1.0,30771-0.0,30771-1.0,30781-0.0,30781-1.0,30791-0.0,30791-1.0,30801-0.0,30801-1.0,30811-0.0,30811-1.0,30821-0.0,30821-1.0,30831-0.0,30831-1.0,30841-0.0,30841-1.0,30851-0.0,30851-1.0,30861-0.0,30861-1.0,30871-0.0,30871-1.0,30881-0.0,30881-1.0,30891-0.0,30891-1.0,53-0.0,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,22700-0.0,22700-0.1,22700-0.2,22700-0.3,22700-0.4,22700-0.5,22700-0.6,22700-0.7,22700-0.8,22700-0.9,22700-0.10,22700-0.11,22700-0.12,22700-0.13,22700-0.14,30532-0.0,30532-1.0,40000-0.0,40000-1.0,40005-0.0,40005-1.0,40005-2.0,40005-3.0,40005-4.0,40005-5.0,40005-6.0,40005-7.0,40005-8.0,40005-9.0,40005-10.0,40005-11.0,40005-12.0,40005-13.0,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0,41262-0.0,41262-0.1,41262-0.2,41262-0.3,41262-0.4,41262-0.5,41262-0.6,41262-0.7,41262-0.8,41262-0.9,41262-0.10,41262-0.11,41262-0.12,41262-0.13,41262-0.14,41262-0.15,41262-0.16,41262-0.17,41262-0.18,41262-0.19,41262-0.20,41262-0.21,41262-0.22,41262-0.23,41262-0.24,41262-0.25,41262-0.26,41262-0.27,41262-0.28,41262-0.29,41262-0.30,41262-0.31,41262-0.32,41262-0.33,41262-0.34,41262-0.35,41262-0.36,41262-0.37,41262-0.38,41262-0.39,41262-0.40,41262-0.41,41262-0.42,41262-0.43,41262-0.44,41262-0.45,41262-0.46,41262-0.47,41262-0.48,41262-0.49,41262-0.50,41262-0.51,41262-0.52,41262-0.53,41262-0.54,41262-0.55,41262-0.56,41262-0.57,41262-0.58,41262-0.59,41262-0.60,41262-0.61,41262-0.62,41262-0.63,41262-0.64,41262-0.65,41262-0.66,41262-0.67,41262-0.68,41262-0.69,41262-0.70,41262-0.71,41262-0.72,41262-0.73,41262-0.74,41263-0.0,41263-0.1,41263-0.2,41263-0.3,41263-0.4,41263-0.5,41263-0.6,41263-0.7,41263-0.8,41263-0.9,41263-0.10,41263-0.11,41263-0.12,41263-0.13,41263-0.14,41263-0.15,41263-0.16,41263-0.17,41263-0.18,41263-0.19,41263-0.20,41263-0.21,41263-0.22,41263-0.23,41263-0.24,41263-0.25,41263-0.26,41263-0.27,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,41280-0.10,41280-0.11,41280-0.12,41280-0.13,41280-0.14,41280-0.15,41280-0.16,41280-0.17,41280-0.18,41280-0.19,41280-0.20,41280-0.21,41280-0.22,41280-0.23,41280-0.24,41280-0.25,41280-0.26,41280-0.27,41280-0.28,41280-0.29,41280-0.30,41280-0.31,41280-0.32,41280-0.33,41280-0.34,41280-0.35,41280-0.36,41280-0.37,41280-0.38,41280-0.39,41280-0.40,41280-0.41,41280-0.42,41280-0.43,41280-0.44,41280-0.45,41280-0.46,41280-0.47,41280-0.48,41280-0.49,41280-0.50,41280-0.51,41280-0.52,41280-0.53,41280-0.54,41280-0.55,41280-0.56,41280-0.57,41280-0.58,41280-0.59,41280-0.60,41280-0.61,41280-0.62,41280-0.63,41280-0.64,41280-0.65,41280-0.66,41280-0.67,41280-0.68,41280-0.69,41280-0.70,41280-0.71,41280-0.72,41280-0.73,41280-0.74,41280-0.75,41280-0.76,41280-0.77,41280-0.78,41280-0.79,41280-0.80,41280-0.81,41280-0.82,41280-0.83,41280-0.84,41280-0.85,41280-0.86,41280-0.87,41280-0.88,41280-0.89,41280-0.90,41280-0.91,41280-0.92,41280-0.93,41280-0.94,41280-0.95,41280-0.96,41280-0.97,41280-0.98,41280-0.99,41280-0.100,41280-0.101,41280-0.102,41280-0.103,41280-0.104,41280-0.105,41280-0.106,41280-0.107,41280-0.108,41280-0.109,41280-0.110,41280-0.111,41280-0.112,41280-0.113,41280-0.114,41280-0.115,41280-0.116,41280-0.117,41280-0.118,41280-0.119,41280-0.120,41280-0.121,41280-0.122,41280-0.123,41280-0.124,41280-0.125,41280-0.126,41280-0.127,41280-0.128,41280-0.129,41280-0.130,41280-0.131,41280-0.132,41280-0.133,41280-0.134,41280-0.135,41280-0.136,41280-0.137,41280-0.138,41280-0.139,41280-0.140,41280-0.141,41280-0.142,41280-0.143,41280-0.144,41280-0.145,41280-0.146,41280-0.147,41280-0.148,4128

In [54]:
# sas로 가져가기

df.to_csv('data\\ukb_date_merged_DATETIME.csv', encoding = 'euc-kr', index = False)

In [35]:
# # datetime 형식인 열들 str로 바꾸기 - 안됨

# df.iloc[:, 67:120] = df.iloc[:, 67:120].strftime("%Y-%m-%d")

AttributeError: 'DataFrame' object has no attribute 'strftime'

In [36]:
# # 왜 안바뀌지 ..? 전체 다 해보자  - DATETIME 형식을 astype로 str로 바꿀 순 없는 듯 하다

# all_columns = df.columns.tolist()
# df[all_columns] = df[all_columns].astype(str)

In [40]:
df.replace({pd.NaT: np.NaN}, inplace = True)

In [41]:
df.iloc[:, 67:120]

,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,...,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-20 00:00:00,NaN,2015-08-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-09-20 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,2011-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,NaN,2009-09-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-09 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 왜 안돼 ....

last_followup_date = np.nanmax(df, axis = 1)

In [13]:
last_followup_date = pd.DataFrame(last_followup_date)

In [14]:
last_followup_date

,0
0,2020-09-18
1,2017-02-05
2,2017-10-19
3,2017-03-08
4,2020-02-13
...,...
502470,2016-05-10
502471,2017-04-07
502472,2015-02-21
502473,2019-07-27


In [16]:
df_final = pd.concat([id_tmp, last_followup_date], axis = 1)
df_final

,eid,0
0,1000015.0,2020-09-18
1,1000027.0,2017-02-05
2,1000039.0,2017-10-19
3,1000040.0,2017-03-08
4,1000053.0,2020-02-13
...,...,...
502470,6025327.0,2016-05-10
502471,6025339.0,2017-04-07
502472,6025340.0,2015-02-21
502473,6025353.0,2019-07-27


In [17]:
df_final.rename(columns = {0 : 'LastFollowUpDate'}, inplace = True)
df_final

,eid,LastFollowUpDate
0,1000015.0,2020-09-18
1,1000027.0,2017-02-05
2,1000039.0,2017-10-19
3,1000040.0,2017-03-08
4,1000053.0,2020-02-13
...,...,...
502470,6025327.0,2016-05-10
502471,6025339.0,2017-04-07
502472,6025340.0,2015-02-21
502473,6025353.0,2019-07-27


In [20]:
len(df_final.LastFollowUpDate.unique())

# 50만명인데 3031..? 이게 맞나? 

3031

In [21]:
# LastFollowUpDate created

df_final.to_csv('data\\ukb_date_LastFollowUpDatepreprocessed.csv', encoding = 'euc-kr', index = False)

In [ ]:
pd.set_option('display.max_column', None)
k = df.iloc[[0]]
k

In [ ]:
np.nanmax(k, axis = 1)

In [ ]:
ary = ['2012-05-31', '2013-03-03', '2016-05-05', np.nan]
ary

In [ ]:
ary

In [ ]:
df['30502-0.0']

#### 2012.12.09 ~
#### ukb46987_new_icd9 에서 icd9 코드로 ckd 진단받은 사람 찾아서 자르기

In [2]:
ukb46987_icd9 = pd.read_sas('SAS\\ukb46987_new_icd9.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
ukb46987_icd9

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,6180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = ukb46987_icd9.copy()

In [5]:
pd.set_option('display.max_column', None)
df

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,41271-0.9,41271-0.10,41271-0.11,41271-0.12,41271-0.13,41271-0.14,41271-0.15,41271-0.16,41271-0.17,41271-0.18,41271-0.19,41271-0.20,41271-0.21,41271-0.22,41271-0.23,41271-0.24,41271-0.25,41271-0.26,41271-0.27,41271-0.28,41271-0.29,41271-0.30,41271-0.31,41271-0.32,41271-0.33,41271-0.34,41271-0.35,41271-0.36,41271-0.37,41271-0.38,41271-0.39,41271-0.40,41271-0.41,41271-0.42,41271-0.43,41271-0.44,41271-0.45,41271-0.46,41281-0.0,41281-0.1,41281-0.2,41281-0.3,41281-0.4,41281-0.5,41281-0.6,41281-0.7,41281-0.8,41281-0.9,41281-0.10,41281-0.11,41281-0.12,41281-0.13,41281-0.14,41281-0.15,41281-0.16,41281-0.17,41281-0.18,41281-0.19,41281-0.20,41281-0.21,41281-0.22,41281-0.23,41281-0.24,41281-0.25,41281-0.26,41281-0.27,41281-0.28,41281-0.29,41281-0.30,41281-0.31,41281-0.32,41281-0.33,41281-0.34,41281-0.35,41281-0.36,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,75310,75311,75312,5859
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [8]:
# df2 : icd9 진단여부
df2 = df.iloc[:, :48]
df2

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,41271-0.9,41271-0.10,41271-0.11,41271-0.12,41271-0.13,41271-0.14,41271-0.15,41271-0.16,41271-0.17,41271-0.18,41271-0.19,41271-0.20,41271-0.21,41271-0.22,41271-0.23,41271-0.24,41271-0.25,41271-0.26,41271-0.27,41271-0.28,41271-0.29,41271-0.30,41271-0.31,41271-0.32,41271-0.33,41271-0.34,41271-0.35,41271-0.36,41271-0.37,41271-0.38,41271-0.39,41271-0.40,41271-0.41,41271-0.42,41271-0.43,41271-0.44,41271-0.45,41271-0.46
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,6180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# df3 : icd9 진단날짜

df3 = df.iloc[:, 48:]
df3

,41281-0.0,41281-0.1,41281-0.2,41281-0.3,41281-0.4,41281-0.5,41281-0.6,41281-0.7,41281-0.8,41281-0.9,41281-0.10,41281-0.11,41281-0.12,41281-0.13,41281-0.14,41281-0.15,41281-0.16,41281-0.17,41281-0.18,41281-0.19,41281-0.20,41281-0.21,41281-0.22,41281-0.23,41281-0.24,41281-0.25,41281-0.26,41281-0.27,41281-0.28,41281-0.29,41281-0.30,41281-0.31,41281-0.32,41281-0.33,41281-0.34,41281-0.35,41281-0.36,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,75310,75311,75312,5859
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
502473,1989-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
502474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


#### icd9 ckd 진단코드 

Unlike the usually harmless simple kidney cysts that can form in the kidneys later in life, PKD cysts can change the shape of your kidneys, including making them much larger. PKD is a form of chronic kidney disease (CKD) that reduces kidney function and may lead to kidney failure.

- 75310 Cystic kidney disease (renal cyst (single))
- 75311 Polycystic kidneys, infantile type
- 75312 Polycystic kidneys, adult type
- 75313 Polycystic kidneys, not otherwise specified  
- 5859 Chronic renal failure

In [4]:
df['75310'] = 0
df['75311'] = 0
df['75312'] = 0
df['5859'] = 0

In [14]:
colnames = df2.columns.tolist()
colnames.remove('eid')

for col in colnames: 
    df2.loc[df2[col] == '75310', '75310'] = 1  # 전체 222개 칼럼 순회하면서 75310 있으면 1 넣어주기
    df2.loc[df2[col] == '75310', '75310_index'] = col[6:]  # 41281 변수에서 75310 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [15]:
for col in colnames: 
    df2.loc[df2[col] == '75311', '75311'] = 1  # 전체 222개 칼럼 순회하면서 75311 있으면 1 넣어주기
    df2.loc[df2[col] == '75311', '75311_index'] = col[6:]  # 41281 변수에서 75311 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [16]:
for col in colnames: 
    df2.loc[df2[col] == '75312', '75312'] = 1  # 전체 222개 칼럼 순회하면서 75312 있으면 1 넣어주기
    df2.loc[df2[col] == '75312', '75312_index'] = col[6:]  # 41281 변수에서 75312 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [17]:
for col in colnames: 
    df2.loc[df2[col] == '5859', '5859'] = 1  # 전체 222개 칼럼 순회하면서 5859 있으면 1 넣어주기
    df2.loc[df2[col] == '5859', '5859_index'] = col[6:]  # 41281 변수에서 5859 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [18]:
print(df2["75310"].value_counts())
print(df2["75310_index"].isna().sum())

Series([], Name: 75310, dtype: int64)
502476


In [19]:
print(df2["75311"].value_counts())
print(df2["75311_index"].isna().sum())

Series([], Name: 75311, dtype: int64)
502476


In [20]:
print(df2["75312"].value_counts())
print(df2["75312_index"].isna().sum())

Series([], Name: 75312, dtype: int64)
502476


In [21]:
print(df2["5859"].value_counts())
print(df2["5859_index"].isna().sum())

1.0    7
Name: 5859, dtype: int64
502469


In [25]:
# ckd 진단자 5859에서만 7명

In [33]:
# 그 7명 id 리스트로 만들기

icd9_ckd_diagnosed = df2.loc[df2['5859'] == 1, 'eid'].tolist()
icd9_ckd_diagnosed

['1614856', '1921916', '1969684', '2889023', '4723626', '5323738', '5761594']

In [34]:
icd10 = pd.read_csv('data\\ukb46987_N18preprocessed.csv')
icd10

,eid,N18_Date,N18
0,1000015,0,0
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [37]:
# icd9 진단받은 7명 최종 데이터프레임에서 자르기

final = icd10.loc[~icd10['eid'].isin(icd9_ckd_diagnosed)].reset_index(drop = True)
final
# 원래 없다

,eid,N18_Date,N18
0,1000015,0,0
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [45]:
# recheck - 원래 없다

icd10.loc[icd10['eid'] == '5761594']

,eid,N18_Date,N18
